In [ ]:
import os
import numpy as np
from PIL import Image
import torchvision.models as models
!pip install adabelief-pytorch==0.1.0
from adabelief_pytorch import AdaBelief
from torch.autograd import Variable
import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
import natsort
from torch.utils.data import Dataset, DataLoader
import gc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
!git clone https://github.com/prajnasb/observations.git

Cloning into 'observations'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 1638 (delta 9), reused 0 (delta 0), pack-reused 1604
Receiving objects: 100% (1638/1638), 75.94 MiB | 8.77 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
data_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224)),                                        
        torchvision.transforms.RandomAffine(20,(0.2,0.2),None,0.15,0,0),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor()
    ])

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root='/content/observations/experiements/dest_folder/train/', 
                                                 transform=data_transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, 
                                               shuffle=True, num_workers=24)

val_dataset = torchvision.datasets.ImageFolder(root='/content/observations/experiements/dest_folder/val/', 
                                               transform=data_transform)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, 
                                             shuffle=False, num_workers=24)
test_dataset = torchvision.datasets.ImageFolder(root='/content/observations/experiements/dest_folder/test/', 
                                               transform=data_transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, 
                                             shuffle=False, num_workers=24)

In [ ]:
def train(model, data_loader, test_loader, task='Classification'):
    model.train()

    for epoch in range(numEpochs):
        avg_loss = 0.0
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats)

            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()

            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0    
            
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
        if task == 'Classification':
            val_loss, val_acc = test_classify(model, test_loader)
            train_loss, train_acc = test_classify(model, data_loader)
            print('Epoch: {}\tTrain Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(epoch+1,train_loss, train_acc, val_loss, val_acc))
        else:
            test_verify(model, test_loader)


def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total
def init_xavier(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [ ]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(output_size=(1,1)),
            nn.Dropout(0.5)
        )
        flatten = self.cnn_layers(torch.rand(1, 3, 224, 224)).view(-1).size(0)
        self.linear_layers = nn.Sequential(
            nn.Linear(flatten, 512),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.Dropout(0.2),
            nn.Linear(256, 2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
network = Net()
print(network)
network.apply(init_xavier)
criterion = nn.CrossEntropyLoss()
optimizer =  AdaBelief(network.parameters(), lr=1e-3, eps=1e-8, betas=(0.9,0.999), weight_decouple = True, rectify = False, weight_decay=1e-2,fixed_decay=False,amsgrad=False)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU(inplace=True)
    (

In [ ]:
numEpochs = 20
network.train()
network.to(device)
train(network, train_dataloader, val_dataloader)

Epoch: 1	Train Loss: 3.0333	Train Accuracy: 0.4996	Val Loss: 3.0186	Val Accuracy: 0.5000
Epoch: 2	Train Loss: 0.2502	Train Accuracy: 0.9202	Val Loss: 0.1564	Val Accuracy: 0.9225
Epoch: 3	Train Loss: 0.1432	Train Accuracy: 0.9658	Val Loss: 0.0635	Val Accuracy: 0.9718
Epoch: 4	Train Loss: 0.1862	Train Accuracy: 0.9346	Val Loss: 0.1151	Val Accuracy: 0.9437
Epoch: 5	Train Loss: 1.4125	Train Accuracy: 0.5703	Val Loss: 1.0924	Val Accuracy: 0.6127
Epoch: 6	Train Loss: 0.1464	Train Accuracy: 0.9536	Val Loss: 0.0226	Val Accuracy: 0.9930
Epoch: 7	Train Loss: 0.0696	Train Accuracy: 0.9810	Val Loss: 0.0328	Val Accuracy: 0.9789
Epoch: 8	Train Loss: 0.0670	Train Accuracy: 0.9787	Val Loss: 0.0238	Val Accuracy: 0.9930
Epoch: 9	Train Loss: 0.0523	Train Accuracy: 0.9833	Val Loss: 0.0132	Val Accuracy: 1.0000
Epoch: 10	Train Loss: 0.2190	Train Accuracy: 0.9156	Val Loss: 0.1231	Val Accuracy: 0.9577
Epoch: 11	Train Loss: 0.4833	Train Accuracy: 0.9042	Val Loss: 0.4995	Val Accuracy: 0.9366
Epoch: 12	Train Los

#The results of this network
            self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(128*26*26, 512),
            nn.Linear(512, 256),
            nn.Linear(256, 2)
        )
#are :
Epoch: 1	Train Loss: 28.7583	Train Accuracy: 0.4996	Val Loss: 26.2200	Val Accuracy: 0.5000

Epoch: 2	Train Loss: 2.4462	Train Accuracy: 0.8738	Val Loss: 0.8310	Val Accuracy: 0.8944

Epoch: 3	Train Loss: 1.5042	Train Accuracy: 0.7559	Val Loss: 1.1459	Val Accuracy: 0.8169

Epoch: 4	Train Loss: 4.3187	Train Accuracy: 0.8829	Val Loss: 2.8687	Val Accuracy: 0.9085

Epoch: 5	Train Loss: 0.6918	Train Accuracy: 0.8700	Val Loss: 0.5393	Val Accuracy: 0.9085

Epoch: 6	Train Loss: 0.1974	Train Accuracy: 0.9285	Val Loss: 0.0879	Val Accuracy: 0.9648

Epoch: 7	Train Loss: 0.2421	Train Accuracy: 0.9262	Val Loss: 0.1064	Val Accuracy: 0.9648

Epoch: 8	Train Loss: 0.1956	Train Accuracy: 0.9354	Val Loss: 0.0992	Val Accuracy: 0.9577

Epoch: 9	Train Loss: 0.1959	Train Accuracy: 0.9521	Val Loss: 0.1169	Val Accuracy: 0.9577

Epoch: 10	Train Loss: 0.1911	Train Accuracy: 0.9361	Val Loss: 0.1109	Val Accuracy: 0.9648
  

#The results of this network            
            self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2),
        )
        flatten = self.cnn_layers(torch.rand(1, 3, 224, 224)).view(-1).size(0)
        self.linear_layers = nn.Sequential(
            nn.Linear(flatten, 512),
            nn.Linear(512, 256),
            nn.Linear(256, 2)
        )
#are : 
Epoch: 1	Train Loss: 3.4300	Train Accuracy: 0.4996	Val Loss: 3.4068	Val Accuracy: 0.5000

Epoch: 2	Train Loss: 0.7142	Train Accuracy: 0.5414	Val Loss: 0.7142	Val Accuracy: 0.5211

Epoch: 3	Train Loss: 0.2586	Train Accuracy: 0.9316	Val Loss: 0.0856	Val Accuracy: 0.9718

Epoch: 4	Train Loss: 0.2069	Train Accuracy: 0.9293	Val Loss: 0.0955	Val Accuracy: 0.9718

Epoch: 5	Train Loss: 0.1440	Train Accuracy: 0.9475	Val Loss: 0.0986	Val Accuracy: 0.9648

Epoch: 6	Train Loss: 0.1119	Train Accuracy: 0.9597	Val Loss: 0.0307	Val Accuracy: 0.9930

Epoch: 7	Train Loss: 0.3444	Train Accuracy: 0.8502	Val Loss: 0.2988	Val Accuracy: 0.8592

Epoch: 8	Train Loss: 0.1023	Train Accuracy: 0.9627	Val Loss: 0.0425	Val Accuracy: 0.9648

Epoch: 9	Train Loss: 0.1170	Train Accuracy: 0.9559	Val Loss: 0.0261	Val Accuracy: 0.9930

Epoch: 10	Train Loss: 0.0862	Train Accuracy: 0.9757	Val Loss: 0.0393	Val Accuracy: 0.9859

Epoch: 11	Train Loss: 0.1210	Train Accuracy: 0.9620	Val Loss: 0.0472	Val Accuracy: 0.9718

Epoch: 12	Train Loss: 0.1233	Train Accuracy: 0.9681	Val Loss: 0.0361	Val Accuracy: 0.9930

Epoch: 13	Train Loss: 1.2096	Train Accuracy: 0.5559	Val Loss: 1.1720	Val Accuracy: 0.5775

Epoch: 14	Train Loss: 0.0783	Train Accuracy: 0.9658	Val Loss: 0.0403	Val Accuracy: 0.9859

Epoch: 15	Train Loss: 0.1185	Train Accuracy: 0.9605	Val Loss: 0.0341	Val Accuracy: 0.9859

Epoch: 16	Train Loss: 0.0794	Train Accuracy: 0.9741	Val Loss: 0.0232	Val Accuracy: 0.9859

Epoch: 17	Train Loss: 0.1391	Train Accuracy: 0.9559	Val Loss: 0.0521	Val Accuracy: 0.9718

Epoch: 18	Train Loss: 0.1867	Train Accuracy: 0.9483	Val Loss: 0.1596	Val Accuracy: 0.9296

Epoch: 19	Train Loss: 0.1882	Train Accuracy: 0.9194	Val Loss: 0.0730	Val Accuracy: 0.9789

Epoch: 20	Train Loss: 0.4546	Train Accuracy: 0.8175	Val Loss: 0.3714	Val Accuracy: 0.8451

#The results of this network(Best one so far)          
            self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(output_size=(1,1)),
            nn.Dropout(0.5)
        )
        flatten = self.cnn_layers(torch.rand(1, 3, 224, 224)).view(-1).size(0)
        self.linear_layers = nn.Sequential(
            nn.Linear(flatten, 512),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.Dropout(0.2),
            nn.Linear(256, 2)
        )
#are :
Epoch: 1	Train Loss: 3.0333	Train Accuracy: 0.4996	Val Loss: 3.0186	Val Accuracy: 0.5000

Epoch: 2	Train Loss: 0.2502	Train Accuracy: 0.9202	Val Loss: 0.1564	Val Accuracy: 0.9225

Epoch: 3	Train Loss: 0.1432	Train Accuracy: 0.9658	Val Loss: 0.0635	Val Accuracy: 0.9718

Epoch: 4	Train Loss: 0.1862	Train Accuracy: 0.9346	Val Loss: 0.1151	Val Accuracy: 0.9437

Epoch: 5	Train Loss: 1.4125	Train Accuracy: 0.5703	Val Loss: 1.0924	Val Accuracy: 0.6127

Epoch: 6	Train Loss: 0.1464	Train Accuracy: 0.9536	Val Loss: 0.0226	Val Accuracy: 0.9930

Epoch: 7	Train Loss: 0.0696	Train Accuracy: 0.9810	Val Loss: 0.0328	Val Accuracy: 0.9789

Epoch: 8	Train Loss: 0.0670	Train Accuracy: 0.9787	Val Loss: 0.0238	Val Accuracy: 0.9930

Epoch: 9	Train Loss: 0.0523	Train Accuracy: 0.9833	Val Loss: 0.0132	Val Accuracy: 1.0000

Epoch: 10	Train Loss: 0.2190	Train Accuracy: 0.9156	Val Loss: 0.1231	Val Accuracy: 0.9577

Epoch: 11	Train Loss: 0.4833	Train Accuracy: 0.9042	Val Loss: 0.4995	Val Accuracy: 0.9366

Epoch: 12	Train Loss: 0.6836	Train Accuracy: 0.8631	Val Loss: 0.6065	Val Accuracy: 0.8380

Epoch: 13	Train Loss: 0.0518	Train Accuracy: 0.9787	Val Loss: 0.0128	Val Accuracy: 1.0000

Epoch: 14	Train Loss: 0.0846	Train Accuracy: 0.9597	Val Loss: 0.0218	Val Accuracy: 1.0000

Epoch: 15	Train Loss: 0.0307	Train Accuracy: 0.9909	Val Loss: 0.0091	Val Accuracy: 1.0000

Epoch: 16	Train Loss: 0.0896	Train Accuracy: 0.9711	Val Loss: 0.0661	Val Accuracy: 0.9930

Epoch: 17	Train Loss: 0.0459	Train Accuracy: 0.9817	Val Loss: 0.0315	Val Accuracy: 0.9930

Epoch: 18	Train Loss: 0.1224	Train Accuracy: 0.9506	Val Loss: 0.0716	Val Accuracy: 0.9789

Epoch: 19	Train Loss: 0.0716	Train Accuracy: 0.9719	Val Loss: 0.0871	Val Accuracy: 0.9648

Epoch: 20	Train Loss: 0.2875	Train Accuracy: 0.9384	Val Loss: 0.3295	Val Accuracy: 0.9507